In [12]:
import os
import collections

from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
import pymorphy2 as pm
import string
import re

import copy

In [1]:
#Создаем словарь, где ключ - название эпизода, значение - текст субтитров.
ep_text_d = collections.defaultdict()
print(ep_text_d)

defaultdict(None, {})


In [4]:
#Выкачаем все файлы
files_dir = os.getcwd()+'\\friends'
for dirt in os.listdir(files_dir):
    for file in os.listdir(files_dir+'\\'+dirt):
        f_name = files_dir + '\\' + dirt + '\\' + file
        with open(f_name, 'r', encoding='utf-8') as f:
            ep_text_d[file] = f.read().replace('\n', ' ')
print(len(ep_text_d.keys()))

165


In [ ]:
#!pip install pymorphy2

In [6]:
#Сделаем предпроцессинг данных. Приведем тексты к нормальным формам, удалим стоплова и знаки препинания. 
stopWords = set(stopwords.words('russian'))
morph = pm.MorphAnalyzer()

def preprocess(text):
    """Функция на вход получает текст и возвращает список нормализованных слов, без знаков пунктуации, без стопслов"""
    text = text.lower()
    tokenizer = RegexpTokenizer(r'\w+')
    tokens = tokenizer.tokenize(text)
    filtered_words = list(filter(lambda token: token not in stopwords.words('russian'), tokens))
    norm_words = [morph.parse(token)[0].normal_form for token in filtered_words]
    return norm_words

def indexing(d):
    """Функция на вход получает словарь эпизод: текст, на выходе словарь эпизод: список лемм в тесте"""
    index_dict = {} 
    for key in d.keys():
        index_dict[key] = preprocess(d[key])
    return index_dict

index_d = indexing(ep_text_d)

In [7]:
def metric(d, key, item):
    """
    Функция считает метрику сходства для нормализованного поискового запроса.
    А именно - Term-frequency.
    """
    if key not in d.keys():
        return 0
    
    tf = d[key].count(item) / len(d[key])
    
    return tf

In [8]:
def reverse_indx(d):
    """
    Функция на вход получает индексированный словарь, на выход - обратный индекс
    Где для каждого токена есть словарь вида tf: эпизод
    """

    reverse_index_dict = {}
    for key, value in d.items():
        for item in value:
            tf = metric(d, key, item)
            if item not in reverse_index_dict.keys():
                #reverse_index_dict[item] = {tf : key}
                reverse_index_dict[item] = {key:tf}
            else:
                #reverse_index_dict[item][tf] = key
                reverse_index_dict[item][key]  = tf
    return reverse_index_dict

reverse_d = reverse_indx(index_d)

In [9]:
reverse_d['марсель']

{'Friends - 1x10 - The One With The Monkey.ru.txt': 0.0036231884057971015,
 'Friends - 1x15 - The One With The Stoned Guy.ru.txt': 0.001034126163391934,
 'Friends - 1x16 - The One With Two Parts (1).ru.txt': 0.003883495145631068,
 'Friends - 1x17 - The One With Two Parts (2).ru.txt': 0.002053388090349076,
 'Friends - 1x18 - The One With All The Poker.ru.txt': 0.0019704433497536944,
 'Friends - 1x19 - The One Where The Monkey Gets Away.ru.txt': 0.022510822510822513,
 'Friends - 2x12 - The One After The Super Bowl (1).ru.txt': 0.006433823529411764,
 'Friends - 2x13 - The One After The Super Bowl (2).ru.txt': 0.006185567010309278,
 'Friends - 2x14 - The One With The Prom Video.ru.txt': 0.0019267822736030828,
 'Friends - 7x17 - The One With The Cheap Wedding Dress.ru.txt': 0.0009182736455463728}

In [10]:
def search(s, reverse_d):
    """Функция выполняет поиск по reverse_d, выводит отсортированный список результатов"""
    
    norm_s = preprocess(s)
    result = {}
    result_sorted=[]
    
    for token in norm_s:
        if token not in reverse_d.keys():
            pass
        else:
            if len(result)  == 0 : 
                result = copy.deepcopy(reverse_d[token])
                #result = reverse_d[token] #episode:tf
            else:
                check = reverse_d[token] #episode:tf
                for key, value in check.items():
                    if key in result.keys():
                        result[key] += check[key]
                    else:
                        result[key] = check[key]
    
    result_inverse = {y:x for x,y in result.items()} 
    for tf in sorted(result_inverse, reverse=True):
        result_sorted.append(result_inverse[tf] + 'TF--{}'.format(tf))
    return result_sorted

In [13]:
#Проверим на 'марсель' - это обезьянка, которая была в первом сезоне в основном
search('марсель', reverse_d)

['Friends - 1x19 - The One Where The Monkey Gets Away.ru.txtTF--0.022510822510822513',
 'Friends - 2x12 - The One After The Super Bowl (1).ru.txtTF--0.006433823529411764',
 'Friends - 2x13 - The One After The Super Bowl (2).ru.txtTF--0.006185567010309278',
 'Friends - 1x16 - The One With Two Parts (1).ru.txtTF--0.003883495145631068',
 'Friends - 1x10 - The One With The Monkey.ru.txtTF--0.0036231884057971015',
 'Friends - 1x17 - The One With Two Parts (2).ru.txtTF--0.002053388090349076',
 'Friends - 1x18 - The One With All The Poker.ru.txtTF--0.0019704433497536944',
 'Friends - 2x14 - The One With The Prom Video.ru.txtTF--0.0019267822736030828',
 'Friends - 1x15 - The One With The Stoned Guy.ru.txtTF--0.001034126163391934',
 'Friends - 7x17 - The One With The Cheap Wedding Dress.ru.txtTF--0.0009182736455463728']

In [14]:
#В 1х19 марсель будет сниматься в кино, попробуем поискать "марсель кино" 
search('марсель кино', reverse_d)

['Friends - 1x19 - The One Where The Monkey Gets Away.ru.txtTF--0.024242424242424246',
 'Friends - 2x13 - The One After The Super Bowl (2).ru.txtTF--0.007216494845360824',
 'Friends - 2x12 - The One After The Super Bowl (1).ru.txtTF--0.006433823529411764',
 'Friends - 1x16 - The One With Two Parts (1).ru.txtTF--0.003883495145631068',
 'Friends - 2x14 - The One With The Prom Video.ru.txtTF--0.0038535645472061657',
 "Friends - 5x17 - The One With Rachel's Inadvertent Kiss.ru.txtTF--0.0038424591738712775",
 'Friends - 1x10 - The One With The Monkey.ru.txtTF--0.0036231884057971015',
 'Friends - 1x18 - The One With All The Poker.ru.txtTF--0.0029556650246305416',
 'Friends - 6x02 - The One Where Ross Hugs Rachel.ru.txtTF--0.0026200873362445414',
 'Friends - 1x15 - The One With The Stoned Guy.ru.txtTF--0.002068252326783868',
 'Friends - 1x17 - The One With Two Parts (2).ru.txtTF--0.002053388090349076',
 "Friends - 7x07 - The One With Ross's Library Book.ru.txtTF--0.0019782393669634025",
 'Fri

In [15]:
#Посчитаем самое частотное слово:
max_count_v = list(reverse_d.keys())[0]
max_count = len(reverse_d[list(reverse_d.keys())[0]])
for key, value in reverse_d.items():
    if len(reverse_d[key]) > max_count:
        max_count_v = key
        max_count = len(reverse_d[key])
max_count_v, max_count

('просто', 165)

In [16]:
#Посчитаем самое редкое слово
min_count_v = list(reverse_d.keys())[0]
min_count = len(reverse_d[list(reverse_d.keys())[0]])
for key, value in reverse_d.items():
    if len(reverse_d[key]) < min_count:
        min_count_v = key
        min_count = len(reverse_d[key])
min_count_v, min_count 

('придурь', 1)

In [17]:
#Какой набор слов есть во всех документах коллекции
list_of_all = []
for key, value in reverse_d.items():
    if len(reverse_d[key]) == len(ep_text_d.keys()):
        list_of_all.append(key)
list_of_all

['просто', 'ты', 'знать', 'хотеть', 'это', 'сказать', 'думать']

In [18]:
#Кто из главных героев статистически самый популярный
names = {'моника':0, 'чендлер':0, 'джо':0, 'рэйчел':0, 'росс':0, 'фиби':0}
for name in names.keys():
    names[name] = len(reverse_d[name])
names
#оказалось, что это росс, но это далеко не факт, т.к. в сериале гороев называют часто не по полным именам, а "рэйч", "фибс", "мон"

{'моника': 145,
 'чендлер': 32,
 'джо': 58,
 'рэйчел': 56,
 'росс': 158,
 'фиби': 141}

In [20]:
#Посмотрим самый популярный сезон у Моники и Чендлера.
#10 элемент

popular = ['моника', 'чендлер']

def most_popular_season(name):    
    count_season = dict()
    episodes = search(name, reverse_d)
    for episode in episodes:
        if episode[10] not in count_season.keys():
            count_season[episode[10]] = 1
        else:
            count_season[episode[10]] += 1
    key_popular = ''
    max_popular = 0
    for key, value in count_season.items():
        if count_season[key] > max_popular:
            key_popular = key
    
    return 'The most popular season for {} is {}'.format(name, key)

for name in popular:
    print(most_popular_season(name)
    
            
    